In [6]:
# Import usual packages
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import uncertainties as unc
import scipy.constants as const
from scipy import stats
from scipy.stats import poisson
from scipy.stats import uniform
from scipy.stats import beta
from scipy.stats import norm
from scipy.optimize import minimize
from tabulate import tabulate
%matplotlib inline

# Rejection Method
The probability density function for beta is 
\begin{equation}
f(x, a, b) = \frac{\Gamma(a+b)x^{a-1}*(1-x)^{b-1}}{\Gamma(a)\Gamma(b)}
\end{equation}
for $0<=x<=1, a>0, b>0$ where $\Gamma$ is the gamma function (scipy.special.gamma)

fill instr

In [ ]:
##### Fill markdown

In [ ]:
## cofde for dist

## Uniform Sampling

In [11]:
# Find the max value of beta (alpha=5.0, beta=3.), denoted beta_max
a = 5
b = 3
dist = beta(a, b)
x_max = (a - 1) / (a + b - 2)
beta_max = dist.pdf(x_max)

# Generate a uniform distribution sample x between [0,1]
x = random.uniform(low=0., high=1.0, size=1000)

# Generate a uniform distribution sample y between [0,beta_max]
y = random.uniform(low=0., high=beta_max, size=1000)

# For each x_i, compute the event weight, w(x_i), and only keep x_i when w(x_i)<y
def keep(N):
    i = 0
    accept = np.zeros(N)
    iterations = 0
    def W(x):
        a = 4
        b = 5
        return beta.pdf(x, a, b)
    while i < N:
        iterations = iterations + 1
        x = random.uniform(low=0., high=1.0, size=1)
        y = random.uniform(low=0., high=beta_max, size=1)
        w = W(x)
        if w >=y:
            accept[i] = x
            i = i + 1
    return accept, iterations

# Generate this sample until accepted events N=1000 events
keep_uniform, iterations_uniform = keep(1000) 


#### Questions

1. What is the event weight formula?

2. What is the total number of samplings (pairs of x, y) required for this MC simulation?

## Gaussian Sampling
repeat steps above but for gaussian distribution sample x

In [13]:
# For each x_i NOW IN GAUSSIAN SAMPLE, compute the event weight, w(x_i), and only keep x_i when w(x_i)<y
def keep_gaussian(N):
    i = 0
    accept = np.zeros(N)
    iterations = 0
    def W(x):
        a = 4
        b = 5
        return beta.pdf(x, a, b)
    while i < N:
        iterations = iterations + 1
        x = random.normal(loc=0.6, scale=0.2, size=1) # for a random sample of x from the gaussian, let mean=0.6 and scale=0.2 to 
        # resemble the beta distribution as closely as possible 
        y = random.uniform(low=0., high=beta_max, size=1)
        w = W(x)
        if w >=y:
            accept[i] = x
            i = i + 1
    return accept, iterations

# Generate this sample until accepted events N=1000 events
keep_norm, iterations_norm = keep_gaussian(1000)         



#### Questions

1. What is your choice of Gaussian Distribution parameter for the importance sample?

2. What is the event weight formula?

3. What is the total number of samplings (pairs of x, y) required for this MC simulation? How is this compared to the uniform sampling?

## Visualization

# Sphere in 3D

insstr

## Uniform Density

## Linear Density

# Bonus
Consider a density that is a unit exponential function of the radius according to $\rho(r)=\frac{1}{\rho_{0}e^{-\rho_{0}r}}$.